# opening a fits

### Há 53 FFIs ao total. Cada FFI possui 84 subfits (85 valores já que um é do header básico). Dentro de cada FFI vou achar o valor de background e modificar a matrizes para se tornarem quadradas. 

In [1]:
#https://archive.stsci.edu/kepler/ffi_display.php
#https://photutils.readthedocs.io/en/stable/background.html

#git fetch --all
#git reset --hard origin/master
# bibliotecas a serem baixadas
%matplotlib inline                           
%config InlineBackend.figure_format='retina'       # Melhora a resolucao do plot
from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))
import matplotlib.pylab as plt                     # Permite fazer plots utilizando o pacote matplotlib
plt.rcParams['figure.figsize'] = (18.0, 18.0)      # Determina o tamanho do plot dentro do ipython notebook
from astropy.io import fits, votable               # Permite carregar arquivos fits e votable
from astropy.table import Table                    # Permite carregar tabelas
from astropy import wcs                            # Importa o pacote World Coordinate System
import numpy as np         

#this program will apply the bias subtraction,
#flat fielding and subtract the gradient background,
#it will also align the images to the first image in the list

#if you use this code, please cite Oelkers et al. 2015, AJ, 149, 50
import scipy
from scipy import stats
import scipy.ndimage as ndimage
import astropy
from astropy.io import fits
from astropy.nddata.utils import Cutout2D
from astropy.wcs import WCS
import math
import time

#libraries for image registration
import FITS_tools
from FITS_tools.hcongrid import hcongrid

#import relevant libraries for a list
import glob, os
from os import listdir
from os.path import isfile, join

#import relevant spline libraries
from scipy.interpolate import Rbf

from astropy.stats import biweight_location

#definir se é sonic ou usuario
#user = 'sonic'
user = 'usuario'

## criar lista de ffis a serem abertos 

### programa em bash
#!/usr/bin/env bash

cd
if [ -a golden.txt ];
	then rm golden.txt
fi

#para printar os files sem os caminhos
#find /home/sonic/Documents/Mestrado/FFIs/8_golden/*.fits -type f -execdir basename '{}' ';'  > /home/sonic/dia_kepler/mychanges/Python/golden.txt

#para printar os files com os caminhos
find /home/sonic/Documents/Mestrado/FFIs/8_golden/*.fits -type f > /home/sonic/dia_kepler/mychanges/Python/golden.txt

find /home/sonic/Documents/Mestrado/FFIs/*.fits -type f > /home/sonic/dia_kepler/mychanges/Python/allffis.txt

In [2]:
# criar uma função def para achar os arquivos


In [3]:
arquivos = !find *.txt
arquivos
arq=[]
for l in arquivos:
    with open(l) as f:
        arq.append(f.read().splitlines())
for y in range(len(arq)):
     arq[y] = np.sort(arq[y])
allffis = arq[0]
golden = arq[1]
#allffis

## Achando background

### Kepler CCD
<img src="CCD_blockdiagram1.jpg">
### Tess CCD
<img src="./tess_ccd.png">

In [4]:
# Rodar apenas uma vez
## consertando as matrizes - cortando linhas e colunas ruins + transformando em matriz quadrada

#from astropy.stats import sigma_clipped_stats
# mean, median, std = sigma_clipped_stats(data, sigma=3.0)
# print((mean, median, std)) 
#from astropy.stats import biweight_location
#print(biweight_location(data)) 
for i in allffis:
    hdul = fits.HDUList()
    hdul.append(fits.PrimaryHDU())
    hdulist = fits.open(i)
    for j in range(1,len(hdulist)):#      cada uma das 84 imagens
        data = hdulist[j].data
        header = hdulist[j].header
        header['NAXIS1']=1100
        header['NAXIS2']=1024
        header['CRPIX1']=546
        header['CRPIX2']=542
        #sky = biweight_location(data)
        data2 = hdulist[j].data[20:1044,12:1112]
        hdul.append(fits.ImageHDU(data2,header = header))

        hdul.writeto('/home/' + user + '/Documents/Mestrado/FFIs/testeall/'+ i.split('/')[-1], overwrite=True)

In [5]:
# # 1 fits com 84 imagens

# user = 'usuario'
# way='/home/' + user + '/Documents/Mestrado/FFIs/8_golden/teste/kplr2009114174833_ffi-cal.fits'
# hdulist = fits.open(way)

# hdul = fits.HDUList()
# hdul.append(fits.PrimaryHDU())
# for j in range(1,len(hdulist)):#      cada uma das 84 imagens
#     data = hdulist[j].data
#     header = hdulist[j].header
#     header['NAXIS1']=1100
#     header['NAXIS2']=1024
#     header['CRPIX1']=546
#     header['CRPIX2']=542
#     #sky = biweight_location(data)
#     data2 = hdulist[j].data[20:1044,12:1112]
#     hdul.append(fits.ImageHDU(data2,header = header))
    
#     hdul.writeto('/home/' + user + '/Documents/Mestrado/FFIs/8_golden/teste/'+ way.split('/')[-1], overwrite=True)

In [6]:
# # 1 CCD

#user = 'usuario'
#hdulist = fits.open('/home/' + user + '/Documents/Mestrado/FFIs/kplr2009322233047_ffi-cal.fits')
# hdulist[1].header[0]
# #print(dict(a).keys())

#j=10
#hdulist[j].header
# hdul = fits.HDUList()
# hdul.append(fits.PrimaryHDU())
# data = hdulist[j].data
# header = hdulist[j].header
# header['NAXIS1']=1100
# header['NAXIS2']=1024
# header['CRPIX1']=545
# header['CRPIX2']=541
# #sky = biweight_location(data)
# data2 = hdulist[j].data[20:1044,12:1112]
# hdul.append(fits.ImageHDU(data2,header = header))
# hdul[1].header

# #     hdul.writeto('/home/' + user + '/Documents/Mestrado/FFIs/8_golden/teste/'+ i.split('/')[-1]+'.fits', overwrite=True)
# # hdulist = fits.open(i)

In [7]:
#hdulist = fits.open('/home/' + user + '/Documents/Mestrado/FFIs/8_golden/teste/kplr2009114174833_ffi-cal.fits')
#print(dict(hdulist[1].header).keys())

In [8]:
#from photutils.datasets import make_100gaussians_image
#data = make_100gaussians_image()

#import matplotlib.pyplot as plt
#from astropy.visualization import SqrtStretch
#from astropy.visualization.mpl_normalize import ImageNormalize
#norm = ImageNormalize(stretch=SqrtStretch())
#plt.imshow(data, norm=norm, origin='lower', cmap='Greys_r')

In [9]:
#####UPDATE INFORMATION HERE####
#DO YOU WANT TO FLAT FIELD AND BIAS SUBTRACT?
biassub = 0 # yes = 1 no = 0 to bias subtract
flatdiv = 0 # yes = 1 no = 0 to flat field
align = 1# yes = 1 no = 0 to align based on coordinates

#useful directories
rawdir = '/home/usuario/Documents/Mestrado/FFIs/8_golden/' #directory with the raw images
cdedir = '/home/usuario/dia_kepler/mychanges/Python' #directory where the code 'lives'
caldir = 'N/A' #directory with the calibration images such as bias & flat
clndir = '/home/usuario/Documents/Mestrado/FFIs/8_golden/clean/'#directory for the cleaned images to be output